In [ ]:
from langchain_openai import ChatOpenAI

import src.config.config as config

print(config)

llm = ChatOpenAI(model=config.OPENAI_MODEL, base_url=config.OPENAI_BASE_URL)


In [ ]:
# print(config)


response = llm.invoke("Why do parrots have colorful feathers?")
print(response)

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver

# 创建检查点 - 这是短期记忆的核心
checkpointer = MemorySaver()

# 创建一个简单的代理（暂时没有工具）
agent = create_agent(
    model=llm,
    tools=[],
    system_prompt="你是一个有帮助的助手。记住用户提供的所有信息。",
    checkpointer=checkpointer,  # 启用记忆
)

In [ ]:
# 测试第一条消息 - 告诉代理我的名字
response1 = agent.invoke(
    {"messages": [{"role": "user", "content": "我叫张三"}]},
    {"configurable": {"thread_id": "user_123"}},  # thread_id 标识对话
)
print("第一轮对话:")
print(response1["messages"][-1].content)

In [ ]:
# 测试第二条消息 - 问代理我叫什么
response2 = agent.invoke(
    {"messages": [{"role": "user", "content": "我叫什么名字？"}]},
    {"configurable": {"thread_id": "user_123"}},  # 使用相同的 thread_id
)

print("\n第二轮对话（应该记得我叫张三）:")
print(response2["messages"][-1].content)

In [ ]:
print("\n完整对话历史:")
for msg in response2["messages"]:
    role = msg.type if hasattr(msg, "type") else "unknown"
    print(f"[{role}]: {msg.content}")

In [ ]:
# 用户2：不同的 thread_id
response_user2 = agent.invoke(
    {"messages": [{"role": "user", "content": "我叫李四"}]},
    {"configurable": {"thread_id": "user_456"}},  # 不同的 thread_id
)

print("\n用户2的对话:")
print(response_user2["messages"][-1].content)

# 回到用户1 - 验证记忆是独立的
response1_again = agent.invoke(
    {"messages": [{"role": "user", "content": "提醒我我叫什么"}]},
    {"configurable": {"thread_id": "user_123"}},  # 回到用户1
)

print("\n回到用户1（应该仍记得是张三）:")
print(response1_again["messages"][-1].content)

In [ ]:
from langchain.tools import tool


@tool
def calculator(expression: str) -> str:
    """执行数学计算。

    参数:
        expression: 要计算的表达式，如"2+2"或"100*5"

    返回:
        计算结果
    """
    try:
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"错误: {str(e)}"


@tool
def get_current_time() -> str:
    """获取当前时间。"""
    from datetime import datetime

    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


@tool
def weather_lookup(city: str) -> str:
    """获取城市天气（模拟数据）。

    参数:
        city: 城市名称

    返回:
        天气信息
    """
    # 模拟天气数据
    weather_data = {
        "北京": "晴，20°C",
        "上海": "多云，22°C",
        "广州": "晴，28°C",
        "杭州": "阴，18°C",
    }
    return weather_data.get(city, f"未知城市: {city}")


# 查看工具定义
print("已定义的工具:")
for tool_obj in [calculator, get_current_time, weather_lookup]:
    print(f"- {tool_obj.name}: {tool_obj.description}")

In [ ]:
# 创建带工具的代理
agent_with_tools = create_agent(
    model=llm,
    tools=[calculator, get_current_time, weather_lookup],
    system_prompt="""你是一个有帮助的助手。
    
你可以使用以下工具：
- calculator: 进行数学计算
- get_current_time: 获取当前时间
- weather_lookup: 查询城市天气

根据用户的请求选择合适的工具。""",
    checkpointer=MemorySaver(),  # 保持记忆能力
)

# 测试工具：做计算
response = agent_with_tools.invoke(
    {"messages": [{"role": "user", "content": "计算 256 乘以 12"}]},
    {"configurable": {"thread_id": "tool_demo"}},
)

print("计算请求结果:")
print(response["messages"][-1].content)

中间件的使用


In [ ]:
from langchain.agents import create_middleware

# 创建一个中间件来增强系统提示
# 根据用户的消息动态调整


def create_context_middleware():
    """创建一个简单的中间件，在调用模型前处理消息"""
    return create_middleware(
        {
            "name": "ContextEnhancer",
            "before_model": lambda state: enhance_messages(state),
        }
    )


def enhance_messages(state):
    """在调用模型前增强消息"""
    # 这是一个简化的例子
    # 在实际应用中，你可以基于消息内容添加上下文
    messages = state.get("messages", [])
    if messages and "天气" in str(messages[-1]):
        # 如果用户问天气，可以添加额外的上下文
        pass
    return {}  # 在这个简化例子中不修改状态


# 注意：完整的中间件实现需要更多配置
# 这里我们演示了概念

In [ ]:
# 创建一个根据用户类型调整提示的代理
def create_specialized_agent(user_level: str):
    """根据用户水平创建不同的代理"""

    if user_level == "beginner":
        system_prompt = """你是一个初学者友好的助手。
        
解释一切都要简单易懂。使用日常语言而不是技术术语。
当用户提问时，先确保他们理解了基础概念。"""

    elif user_level == "expert":
        system_prompt = """你是一个面向专家的技术助手。
        
提供详细的技术细节。可以假设用户理解复杂概念。
直接给出结论，无需过度解释。"""

    else:
        system_prompt = "你是一个有帮助的助手。"

    return create_agent(
        model=llm,
        tools=[calculator, weather_lookup],
        system_prompt=system_prompt,
        checkpointer=MemorySaver(),
    )


# 创建两个代理，演示不同的上下文
beginner_agent = create_specialized_agent("beginner")
expert_agent = create_specialized_agent("expert")

# 测试初学者版本
response_beginner = beginner_agent.invoke(
    {"messages": [{"role": "user", "content": "什么是API？"}]},
    {"configurable": {"thread_id": "beginner"}},
)

print("初学者友好的解释:")
print(response_beginner["messages"][-1].content[:200] + "...")

In [ ]:
# 测试专家版本
response_expert = expert_agent.invoke(
    {"messages": [{"role": "user", "content": "什么是API？"}]},
    {"configurable": {"thread_id": "expert"}}
)

print("\n专家级的解释:")
print(response_expert["messages"][-1].content[:200] + "...")